## Part 1: Data preparation

In this notebook, we will learn how to extract different kind of object models from images, and how to prepare them for subsequent shape analysis.

In [ ]:
import os
import numpy as np
import imageio
import h5py
import matplotlib.pyplot as plt
import copy
from spline_models.spline_curve_model import spline_curve

### 0. Contour extraction

In the exercises, we will be using two image sets:

- A dataset of brightfield microscopy images of a C. elegans live/dead assay from https://bbbc.broadinstitute.org/BBBC010

In [ ]:
# Visualize a sample of the data by running the lines below
live_worms = imageio.imread('data/C. elegans/1649_1109_0003_Amp5-1_B_20070424_B07_w2_F3BB43AA-D215-44C1-B2FD-A89741699E5F.tif')
dead_worms = imageio.imread('data/C. elegans/1649_1109_0003_Amp5-1_B_20070424_A23_w2_71C5704E-F745-4D29-B3B2-51216C02A45D.tif')

fig, ax = plt.subplots(1, 2)
ax[0].imshow(live_worms, cmap='gray')
ax[0].axis('off')
ax[0].set_title('Live C. elegans')

ax[1].imshow(dead_worms, cmap='gray')
ax[1].axis('off')
ax[1].set_title('Dead C. elegans')

plt.show()

- A dataset of fluorescence microscopy images of wild-type (LMNA++) and lamin-deficient (LMNA--) mouse embryonic fibroblasts (MEF) from https://github.com/kukionfr/Micropattern_MEF_LMNA_Image

In [ ]:
# Visualize a sample of the data by running the lines below
wt_cell = imageio.imread('data/MEF_LMNA/wildtype/raw_images/xy001c1.tif')
wt_nuc = imageio.imread('data/MEF_LMNA/wildtype/raw_images/xy001c2.tif')
lmna_cell = imageio.imread('data/MEF_LMNA/lmna_deficient/raw_images/xy001c1.tif')
lmna_nuc = imageio.imread('data/MEF_LMNA/lmna_deficient/raw_images/xy001c2.tif')

fig, ax = plt.subplots(1, 2)
ax[0].imshow(wt_cell, cmap='Greens')
ax[0].imshow(wt_nuc, cmap='Reds', alpha=0.5)
ax[0].axis('off')
ax[0].set_title('Wild-type')

ax[1].imshow(lmna_cell, cmap='Greens')
ax[1].imshow(lmna_nuc, cmap='Reds', alpha=0.5)
ax[1].axis('off')
ax[1].set_title('Lamin deficient')

plt.show()

**0.1** Load a sample segmentation masks for each datasets with the lines below. 

Notice that the segmentation masks are provided in two different format that are commonly encountered when dealing with microscopy image sets: binary png images or hdf5 file. Which format do you find the most convenient? Why?

In [ ]:
# C. elegans dataset
filename_png = 'data/C. elegans/A01_01_ground_truth.png'
celegans_mask=imageio.imread(filename_png)

In [ ]:
# MEF dataset
filename_h5 = 'data/MEF_LMNA/xy001c1_instances.h5'
with h5py.File(filename_h5, 'r') as h5file:
    mef_mask = h5file['exported_data'][:, :, 0]

**0.2** Visualize the masks from each collection with the lines below. Which immediate difference do you notice between the two segmentation masks? 

In [ ]:
# C. elegans dataset
plt.imshow(celegans_mask)
plt.axis('off')
plt.show()

In [ ]:
# MEF dataset
plt.imshow(mef_mask)
plt.axis('off')
plt.show()

**0.3 [FOR MLF ONLY]**: retreive the set of contour points of each individual objects in the `mef_mask` image.

*Hint: use the `segmentation.boundaries` and `measure.find_contours` function from the `skimage` library.*

In [ ]:
from skimage.measure import find_contours
from skimage.segmentation.boundaries import find_boundaries

In [ ]:
# Desired output: mef_contours array containing the contour points of each object in mef_mask
# Add your code here!
mef_contours = 

**0.4 [FOR MLF ONLY]** Run the lines below to visualize the result.

In [ ]:
# Visualize the result 
fig, ax = plt.subplots(1,len(mef_contours),figsize=(len(mef_contours),1))
for i in range(0,len(mef_contours)):
    obj=np.array(mef_contours[i])
    ax[i].scatter(obj[:,0],obj[:,1],s=.5)
    ax[i].axis('off')

In [ ]:
# Visualize the result on top of the mask data
plt.imshow(mef_mask)
for i in range(0,len(mef_contours)):
    obj=np.array(mef_contours[i])
    plt.scatter(obj[:,1],obj[:,0],s=.5,c='red')
plt.axis('off')
plt.show()

**0.3 [FOR C. elegans ONLY]**: retreive the skeleton of the worm in `celegans_mask`.

*Hint: use the `skeletonize` function provided below.*

In [ ]:
from skimage.morphology import skeletonize_3d
from skimage.measure import label

In [ ]:
def order(source, points):
    ordered=[]
    
    current=source
    while len(points)>0:
        dist=[np.linalg.norm(current-points[i]) for i in range(len(points))]
        ind_next=np.argmin(dist)
        current=copy.deepcopy(points[ind_next])
        ordered+=[current]
        points=np.delete(points,ind_next,axis=0)
    
    return np.array(ordered)

In [ ]:
def skeletonize(img):
    skeletonized=skeletonize_3d(img)
    points=np.transpose(np.where(skeletonized>0))

    for p in range(len(points)):
        x, y = points[p,:]
        square=copy.deepcopy(skeletonized[np.max((0,x-1)):np.min((x+2,skeletonized.shape[0])),np.max((0,y-1)):np.min((y+2,skeletonized.shape[1]))])
        square[1,1]=0
        
        source=[]
        if np.max(label(square))==1:
            source=[x,y]
            break
        if len(source)<1:
            source=points[0]
    
    return order(source, points)

In [ ]:
# Desired output: celegans_contour array containing the contour points of the object in celegans_mask
# Add your code here!
celegans_contour = 

**0.4 [FOR C. elegans ONLY]** Run the lines below to visualize the result.

In [ ]:
# Visualize the result on top of the mask data
plt.scatter(celegans_contour[:,0],celegans_contour[:,1],s=5)
plt.axis('off')
plt.show()

In [ ]:
# Visualize the result on top of the mask data
plt.imshow(celegans_mask)
plt.scatter(celegans_contour[:,1],celegans_contour[:,0],s=1,c='red')
plt.axis('off')
plt.show()

### 1. Discrete (point-based) model

**1.1** Pick one of the objects from either of the datasets and extract a set of N equidistant points on it. How small can N be in order for the point-based model to accurately capture the shape of the object? How does it compare to the amount of points the contour was initially composed of?

In [ ]:
# Choose an integer value for N
N=

# Choose whether you want to work with the MEF or C. elegans dataset
# MEF
#obj=mef_contours[0]
#mask_img=mef_mask

# C. elegans
#obj=celegans_contour
#mask_img=celegans_mask

In [ ]:
# Desired output: point_model array containing N equidistant points on the object contour
# Add your code here!
point_model=

**1.2** Run the lines below to visualize the result.

In [ ]:
# Visualize the result
plt.scatter(point_model[:,0],point_model[:,1],s=10)
plt.axis('off')
plt.show()

In [ ]:
# Visualize the result on top of the mask data
plt.imshow(mask_img)
plt.scatter(point_model[:,1],point_model[:,0],s=1,c='red')
plt.axis('off')
plt.show()

### 2. Continuous (spline-based) model

**2.1** The lines below create a continuous cubic B-spline model from the N points you sampled in 1.1. 

Go back to 1.1 and draw a smaller number of N points from the contour. How small can N be in order for the spline-based model to accurately capture the shape of the object? How does it differ from what you observed in 1.1?

In [ ]:
# Adapt the closed_curve flag depending on whether you chose to work with the C. elegans or MEF data in 1.1
closed_curve=False 
spline_model=spline_curve(N, closed_curve)
spline_model.get_spline_coefficients(point_model)

**2.2** Run the lines below to visualize the result.

In [ ]:
# Since the spline-based model is continuous, one can draw as much points as one wants from it for visualization purpose
spline_model_points=spline_model.sample(1000)

plt.scatter(spline_model_points[:,0],spline_model_points[:,1],s=1)
plt.axis('off')
plt.show()

In [ ]:
plt.imshow(mask_img)
plt.scatter(spline_model_points[:,1],spline_model_points[:,0],s=1,c='red')
plt.axis('off')
plt.show()